In [ ]:
#Setup Gdrive file download extention 
!conda install -y gdown

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import random
import concurrent.futures
from PIL import Image, ImageFilter, ImageEnhance
import numpy as np

def apply_noise(image):
    noise_factor = round(random.uniform(4.5, 5.5), 1)
    print("noise", noise_factor)
    img_array = np.array(image)
    noise = np.random.normal(loc=0, scale=5.0, size=img_array.shape)
    noisy_image = Image.fromarray(np.clip(img_array + noise, 0, 255).astype(np.uint8))
    return noisy_image

def apply_gaussian_blur(image):
    blur_radius = round(random.uniform(4.5, 5.5), 1)
    print("blur", blur_radius)
    return image.filter(ImageFilter.GaussianBlur(5.0))

def apply_brightness(image):
    factor = round(random.uniform(1.2, 1.5), 1)
    print("factor br", factor)
    enhancer = ImageEnhance.Brightness(image)
    bright_image = enhancer.enhance(1.4)
    return bright_image

def apply_darkness(image):
    factor = round(random.uniform(0.5, 0.7), 1)
    print("dark", factor)
    enhancer = ImageEnhance.Brightness(image)
    dark_image = enhancer.enhance(0.6)
    return dark_image

def augment_and_save_image(input_folder, output_folder, filename):
    image_path = os.path.join(input_folder, filename)
    original_image = Image.open(image_path)

    # Apply augmentation
    noisy_image = apply_noise(original_image)
    blurred_image = apply_gaussian_blur(original_image)
    bright_image = apply_brightness(original_image)
    dark_image = apply_darkness(original_image)

    # Resize images to 320x320
    noisy_image = noisy_image.resize((1024, 1024))
    blurred_image = blurred_image.resize((1024, 1024))
    bright_image = bright_image.resize((1024, 1024))
    dark_image = dark_image.resize((1024, 1024))

    # Save augmented images
    base_name, ext = os.path.splitext(filename)
    original_image.save(os.path.join(output_folder, f'{base_name}_org{ext}'))

    noisy_image.save(os.path.join(output_folder, f'{base_name}_noisy{ext}'))
    blurred_image.save(os.path.join(output_folder, f'{base_name}_blurred{ext}'))
    bright_image.save(os.path.join(output_folder, f'{base_name}_bright{ext}'))
    dark_image.save(os.path.join(output_folder, f'{base_name}_dark{ext}'))
    print("done")

def augment_images(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    with concurrent.futures.ThreadPoolExecutor() as executor:
        image_files = [filename for filename in os.listdir(input_folder) if filename.endswith(('.jpg', '.jpeg', '.png'))]
        futures = [executor.submit(augment_and_save_image, input_folder, output_folder, filename) for filename in image_files]

        for future in concurrent.futures.as_completed(futures):
            try:
                future.result()
            except Exception as e:
                print(f"An error occurred: {e}")

# Example usage
input_folder = '/content/train/Train/glaucoma'
output_folder = '/content/drive/MyDrive/augmented_image-10-march/glaucoma'
augment_images(input_folder, output_folder)


In [ ]:
import os

# Get the path to the folder containing the files.
folder_path = '/content/train/Train/normal'

# Get a list of all files in the folder.
files = os.listdir(folder_path)

# Delete the first 2000 files.
for file in files[:2000]:
    file_path = os.path.join(folder_path, file)
    os.remove(file_path)

In [ ]:
# !gdown --id <File ID>
# # https://drive.google.com/drive/folders/14KcDLZPavwCDG9QIvuhcoM9mOE6QrHT9?usp=sharing
# # https://drive.google.com/file/d/1FuI9tD0WtnizMQVKiOdWrCHOO1TW8MKk/view?usp=sharing
# # https://drive.google.com/file/d/1ZYEczEoCGDhgZrxOOTUhYkDvnxTMNnuq/view?usp=sharing
# https://drive.google.com/file/d/1MY8jlBZymLCa6IfDYQxS57d6-umdowwd/view?usp=drive_link
# !gdown --id 1MY8jlBZymLCa6IfDYQxS57d6-umdowwd

In [ ]:
# https://drive.google.com/file/d/1MY8jlBZymLCa6IfDYQxS57d6-umdowwd/view?usp=drive_link
!gdown --id 1ZYEczEoCGDhgZrxOOTUhYkDvnxTMNnuq

In [ ]:
import zipfile

with zipfile.ZipFile('/kaggle/working/my_folder.zip', 'r') as zip_ref:
    zip_ref.extractall('Train/')


In [4]:
import os

# Directory path
directory = '/kaggle/working/Train/content/drive/MyDrive/augmented_image-10-march/glaucoma'
directory_normal = '/kaggle/working/Train/content/drive/MyDrive/augmented_image-10-march/normal'

# Get list of files in the directory
files = os.listdir(directory)

# Count the number of files
num_files = len(files)
# Get list of files in the directory
files_norm = os.listdir(directory_normal)

# Count the number of files
num_files_norm = len(files_norm)
print("Number of files in the directory:", num_files)
print("Number of files in the directory_norm:", num_files_norm)


Number of files in the directory: 4920
Number of files in the directory_norm: 6225


In [ ]:
import os
import random
from shutil import copyfile
from sklearn.model_selection import train_test_split

def split_and_save_dataset(input_folder, output_folder, train_ratio=0.7, val_ratio=0.1, test_ratio=0.2):
    # Create output folders for each class
    train_folder = os.path.join(output_folder, 'train')
    val_folder = os.path.join(output_folder, 'val')
    test_folder = os.path.join(output_folder, 'test')

    for folder in [train_folder, val_folder, test_folder]:
        os.makedirs(os.path.join(folder, 'normal'), exist_ok=True)
        os.makedirs(os.path.join(folder, 'glaucoma'), exist_ok=True)

    # Get a list of all image file paths in the input folder for each class
    normal_paths = [os.path.join(input_folder, 'normal', filename) for filename in os.listdir(os.path.join(input_folder, 'normal')) if filename.endswith(('.jpg', '.jpeg', '.png'))]
    glaucoma_paths = [os.path.join(input_folder, 'glaucoma', filename) for filename in os.listdir(os.path.join(input_folder, 'glaucoma')) if filename.endswith(('.jpg', '.jpeg', '.png'))]

    # Shuffle the lists to randomize the order
    random.shuffle(normal_paths)
    random.shuffle(glaucoma_paths)

    # Split the dataset for each class
    normal_train, normal_test = train_test_split(normal_paths, test_size=test_ratio, random_state=42)
    normal_train, normal_val = train_test_split(normal_train, test_size=val_ratio / (train_ratio + val_ratio), random_state=42)

    glaucoma_train, glaucoma_test = train_test_split(glaucoma_paths, test_size=test_ratio, random_state=42)
    glaucoma_train, glaucoma_val = train_test_split(glaucoma_train, test_size=val_ratio / (train_ratio + val_ratio), random_state=42)

    # Copy images to respective folders
    for path in normal_train:
        copyfile(path, os.path.join(train_folder, 'normal', os.path.basename(path)))

    for path in normal_val:
        copyfile(path, os.path.join(val_folder, 'normal', os.path.basename(path)))

    for path in normal_test:
        copyfile(path, os.path.join(test_folder, 'normal', os.path.basename(path)))

    for path in glaucoma_train:
        copyfile(path, os.path.join(train_folder, 'glaucoma', os.path.basename(path)))

    for path in glaucoma_val:
        copyfile(path, os.path.join(val_folder, 'glaucoma', os.path.basename(path)))

    for path in glaucoma_test:
        copyfile(path, os.path.join(test_folder, 'glaucoma', os.path.basename(path)))



In [ ]:
input_folder = '/kaggle/working/Train/content/drive/MyDrive/augmented_image-10-march'
output_folder = '/kaggle/working/Train_split'
# train_ratio = 0.7
# val_ratio = 0.2
# test_ratio = 0.1

In [ ]:
split_and_save_dataset(input_folder, output_folder, train_ratio=0.7, val_ratio=0.1, test_ratio=0.2)


In [1]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8x-cls.yaml')  # build a new model from YAML
model = YOLO('yolov8x-cls.pt')  # load a pretrained model (recommended for training)
model = YOLO('yolov8x-cls.yaml').load('yolov8x-cls.pt')  # build from YAML and transfer weights

# Train the model
results = model.train(data='/kaggle/working/Train_split', epochs=20, imgsz=320,batch=32,patience=5, optimizer='SGD',dropout=0.5,device=0,lr0=0.001)

YOLOv8x-cls summary: 183 layers, 57422840 parameters, 57422840 gradients, 155.3 GFLOPs
YOLOv8x-cls summary: 183 layers, 57422840 parameters, 57422840 gradients, 155.3 GFLOPs
Transferred 302/302 items from pretrained weights
Ultralytics YOLOv8.2.3 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolov8x-cls.yaml, data=/kaggle/working/Train_split, epochs=20, time=None, patience=5, batch=32, imgsz=320, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.5, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=Fals

2024-04-27 16:32:16,869	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-04-27 16:32:17,667	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-04-27 16:32:18.351544: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-27 16:32:18.351604: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-27 16:32:18.353100: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLA

Overriding model.yaml nc=1000 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1   1844480  ultralytics.nn.modules.conv.Conv             [320, 640, 3, 2]              
  6                  -1  6  13117440  ultralytics.nn.modules.block.C2f             [640, 640, 6, True]           
  7                  -1  1   7375360  ultralyti

wandb: Currently logged in as: shje494 (https-wandb-ai-background-curves-svg-2). Use `wandb login --relogin` to force relogin


AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


...

                   all      0.953          1



20 epochs completed in 0.949 hours.
Optimizer stripped from runs/classify/train2/weights/last.pt, 112.5MB
Optimizer stripped from runs/classify/train2/weights/best.pt, 112.5MB

Validating runs/classify/train2/weights/best.pt...
Ultralytics YOLOv8.2.3 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8x-cls summary (fused): 133 layers, 56125762 parameters, 0 gradients, 153.8 GFLOPs
train: /kaggle/working/Train_split/train... found 7800 images in 2 classes ✅ 
val: /kaggle/working/Train_split/val... found 1116 images in 2 classes ✅ 
test: /kaggle/working/Train_split/test... found 2229 images in 2 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 18/18 [00:12<00:00,  1.41it/s]


                   all      0.953          1
Speed: 0.1ms preprocess, 4.1ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train2
Results saved to runs/classify/train2


lr/pg0,█▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr/pg1,▃▆██▇▇▆▆▆▅▅▄▄▃▃▃▂▂▁▁
lr/pg2,▃▆██▇▇▆▆▆▅▅▄▄▃▃▃▂▂▁▁
metrics/accuracy_top1,▁▂▃▅▅▆▆▇▇▇▇█████████
metrics/accuracy_top5,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/loss,█▇▇▆▅▅▄▄▃▃▃▂▂▂▁▁▁▁▁▁
val/loss,█▇▆▅▄▄▃▃▃▂▂▂▂▁▁▁▁▁▁▁
lr/pg0,6e-05
